In [1]:
# %load_ext autoreload

# %autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

In [3]:
import seqrep.datasets as dset
from seqrep.models import NegativeSamplingLoss, SGNS

### Data

In [15]:
%%time

variant = 'ml-dev-small'

movielens_dataset = dset.MovieLensUserItemDataset(variant)

CPU times: user 371 ms, sys: 58.5 ms, total: 430 ms
Wall time: 433 ms


In [16]:
movielens_dataset.df.groupby('user').size().describe()

count     610.000000
mean      147.990164
std       216.882426
min        12.000000
25%        34.000000
50%        68.000000
75%       159.000000
max      2132.000000
dtype: float64

In [17]:
movielens_dataloader = DataLoader(movielens_dataset, batch_size=2)

In [18]:
%%time

for batch_num, (anchors, targets) in enumerate(movielens_dataloader, start=0):
    print('batch', batch_num + 1, '| anchors:', len(anchors), anchors, ' | targets:', len(targets), targets)
    if batch_num == 4:
        break

batch 1 | anchors: 2 tensor([6614, 6429])  | targets: 2 tensor([7369, 7055])
batch 2 | anchors: 2 tensor([1873, 5918])  | targets: 2 tensor([1947, 5912])
batch 3 | anchors: 2 tensor([4138, 1427])  | targets: 2 tensor([6316, 1926])
batch 4 | anchors: 2 tensor([ 935, 1851])  | targets: 2 tensor([ 998, 2274])
batch 5 | anchors: 2 tensor([4116, 6872])  | targets: 2 tensor([3526, 5986])
CPU times: user 115 ms, sys: 6.86 ms, total: 121 ms
Wall time: 121 ms


In [19]:
model = SGNS(num_embeddings=movielens_dataset.num_items, embedding_dim=10, nn_embedding_kwargs={'sparse': True})

In [20]:
anchor_embeddings, target_embeddings, negative_embeddings = model.forward(anchors, targets)

In [21]:
anchor_embeddings.size()

torch.Size([2, 10])

In [22]:
model.as_embedding(anchors[0].item()) == anchor_embeddings[0, :]

tensor([True, True, True, True, True, True, True, True, True, True])